Họ và tên: Huỳnh Thiện Tùng

MSSV: 19522492

Bài tập: LAB 04

# Import các thư viện cần thiết

In [1]:
import gym
import numpy as np
import time
from IPython import display

#### Khởi tạo môi trường FrozenLake-v0

In [2]:
env = gym.make('FrozenLake-v0')

In [3]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

Không gian trạng thái quan sát: 16

In [4]:
env.observation_space.n

16

Không gian hành động: 4

In [5]:
env.action_space.n

4

#### Cài đặt thuật toán Value Iteration

In [6]:
def value_iteration(env, max_iters, gamma):
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Calculate value of state
        for state in range(env.observation_space.n):
            q_values = []
            # Calculate q-value for each action
            for action in range(env.action_space.n):
                q_value = 0
                # Loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # Get the best action
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # If convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [7]:
v_values = value_iteration(env, max_iters=1000, gamma=0.9)

Converged at 79-th iteration.


In [8]:
v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

#### Cài đặt phương thức policy extraction

In [9]:
def policy_extraction(env, v_values, gamma=0.9):
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    for state in range(env.observation_space.n):
        q_values = []
        # Calculate q_value for each action
        for action in range(env.action_space.n):
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            q_values.append(q_value)
        
        # Get the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [10]:
policy = policy_extraction(env, v_values, gamma=0.9)
policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])

#### Cài đặt thuật toán Policy Iteration

In [11]:
def policy_iteration(env, max_iters, gamma):

    # Initialization
    ini_pi = np.array([env.action_space.sample() for i in range(env.observation_space.n)])

    for i in range(max_iters):
        # Policy Evaluation
        v_values = np.zeros(env.observation_space.n)

        for j in range(max_iters):
            prev_v_values = np.copy(v_values)

            # Calculate value of state
            for state, action in enumerate(ini_pi):
                # Calculate q-value for each action
                q_value = 0
                # Loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                v_values[state] = q_value
          
            # Check for convergence
            if np.all(np.isclose(v_values, prev_v_values)):
                break

        # Policy Improvement
        prev_pi = np.copy(ini_pi)
        for state in range(env.observation_space.n):
            q_values = []
            # Calculate q-value for each action
            for action in range(env.action_space.n):
                q_value = 0
                # Loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * v_values[next_state])
            
                q_values.append(q_value)

            # Get the best action
            best_action = np.argmax(q_values)
            ini_pi[state] = best_action
            
        # If convergence
        if np.all(np.isclose(ini_pi, prev_pi)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return ini_pi

In [12]:
pi = policy_iteration(env, max_iters=1000, gamma=0.9)
pi

Converged at 3-th iteration.


array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])

In [13]:
def play(env, policy):
    state = env.reset()
    total_reward = 0
    done = False
    steps = 0
    #time.sleep(1)
    
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        state = next_state

    return total_reward

In [14]:
play(env, policy)

1.0

In [15]:
def play_multiple_times(env, policy, max_episodes):
    success = 0

    for i in range(max_episodes):
        reward = play(env, policy)

        if reward > 0:
            success += 1
    
    return success

Cho chạy 1000 lần thì có khoảng 743 lần thành công, tuy nhiên chúng ta cần con số chính xác hơn

In [19]:
play_multiple_times(env, policy, 1000)

743

Cho chạy 2000 episodes, mỗi eposodes chạy 1000 vòng lặp

In [20]:
# Initilize parameters
MAX_ITERS = 1000
MAX_EPISODES = 2000
GAMMA = 0.9

#### Thực nghiệm trên môi trường FrozenLake-v0

In [21]:
env = gym.make('FrozenLake-v0')

vi_value = value_iteration(env, max_iters=MAX_ITERS, gamma=GAMMA)
policy_from_value = policy_extraction(env, vi_value, GAMMA)
start_vi = time.time()
vi_number_of_successes = play_multiple_times(env, policy=policy_from_value, max_episodes=MAX_EPISODES)
vi_time = time.time() - start_vi

pi = policy_iteration(env, max_iters=MAX_ITERS, gamma=GAMMA)
start_pi = time.time()
pi_number_of_successes = play_multiple_times(env, policy=pi, max_episodes=MAX_EPISODES)
pi_time = time.time() - start_pi

print(f'Number of successes of Value Iteration in FrozenLake8x8-v0 : {vi_number_of_successes}/{MAX_EPISODES}, Average time : {vi_time/MAX_EPISODES}s')
print(f'Number of successes of Policy Iteration in FrozenLake8x8-v0 : {pi_number_of_successes}/{MAX_EPISODES}, Average time : {pi_time/MAX_EPISODES}s')

Converged at 79-th iteration.
Converged at 5-th iteration.
Number of successes of Value Iteration in FrozenLake8x8-v0 : 1453/2000, Average time : 0.0007138723134994507s
Number of successes of Policy Iteration in FrozenLake8x8-v0 : 1443/2000, Average time : 0.0006920249462127686s


#### Thực nghiệm trên FrozenLake8x8-v0

In [22]:
env = gym.make('FrozenLake8x8-v0')

vi_value = value_iteration(env, max_iters=MAX_ITERS, gamma=GAMMA)
policy_from_value = policy_extraction(env, vi_value, GAMMA)
start_vi = time.time()
vi_number_of_successes = play_multiple_times(env, policy=policy_from_value, max_episodes=MAX_EPISODES)
vi_time = time.time() - start_vi

pi = policy_iteration(env, max_iters=MAX_ITERS, gamma=GAMMA)
start_pi = time.time()
pi_number_of_successes = play_multiple_times(env, policy=pi, max_episodes=MAX_EPISODES)
pi_time = time.time() - start_pi

print(f'Number of successes of Value Iteration in FrozenLake8x8-v0 : {vi_number_of_successes}/{MAX_EPISODES}, Average time :  {vi_time/MAX_EPISODES}s')
print(f'Number of successes of Policy Iteration in FrozenLake8x8-v0 : {pi_number_of_successes}/{MAX_EPISODES}, Average time : {pi_time/MAX_EPISODES}s')

Converged at 117-th iteration.
Converged at 3-th iteration.
Number of successes of Value Iteration in FrozenLake8x8-v0 : 1470/2000, Average time :  0.0012719990015029907s
Number of successes of Policy Iteration in FrozenLake8x8-v0 : 1433/2000, Average time : 0.0013079802989959717s


#### Thực nghiệm trên Taxi-v3

In [23]:
env = gym.make('Taxi-v3')

vi_value = value_iteration(env, max_iters=MAX_ITERS, gamma=GAMMA)
policy_from_value = policy_extraction(env, vi_value, GAMMA)
start_vi = time.time()
vi_number_of_successes = play_multiple_times(env, policy=policy_from_value, max_episodes=MAX_EPISODES)
vi_time = time.time() - start_vi

pi = policy_iteration(env, max_iters=MAX_ITERS, gamma=GAMMA)
start_pi = time.time()
pi_number_of_successes = play_multiple_times(env, policy=pi, max_episodes=MAX_EPISODES)
pi_time = time.time() - start_pi

print(f'Number of successes of Value Iteration in FrozenLake8x8-v0 : {vi_number_of_successes}/{MAX_EPISODES}, Average time : {vi_time/MAX_EPISODES}s')
print(f'Number of successes of Policy Iteration in FrozenLake8x8-v0 : {pi_number_of_successes}/{MAX_EPISODES}, Average time : {pi_time/MAX_EPISODES}s')

Converged at 116-th iteration.
Converged at 17-th iteration.
Number of successes of Value Iteration in FrozenLake8x8-v0 : 2000/2000, Average time : 0.0003414106369018555s
Number of successes of Policy Iteration in FrozenLake8x8-v0 : 2000/2000, Average time : 0.0003098808526992798s


#### Nhận xét

Từ kết quả chạy thục nghiệm ta có thể thấy rằng kết quả về số ván thắng lẫn score của 3 map trên với 2 loại giải thuật Value Iteration và Policy Iteration là có sự chênh lệch khá rõ rệt. Giải thuật Policy Iteration hội tụ nhanh hơn so với giải thuật Value Iteration và thời gian chạy cũng nhanh hơn tuy nhiên sự chênh lệch không lớn lắm đối với 2000 EPISODES. 

Xét về độ góc độ cài đặt, Policy Iteration nhìn chung có vẻ phức tạp hơn nhưng chạy nhanh hơn so với Value Iteration. Cả hai giải thuật này đều bảo đảm rằng sẽ hội tụ tới một chiến lược tối ưu, tuy nhiên hai giải thuật này có một số đặc điểm khác biệt rõ về mặc cài đặc giải thuật, chi phí tính toán, tốc độ thực thi,...